# Sandra Package
[Sandra Package on PyPI](https://pypi.org/project/Sandra/)

In [ ]:
!pip install Sandra==0.0.18

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 13.1 MB/s eta 0:00:00


In [ ]:
import Sandra
import gdown
import pandas as pd
from google.colab import files
import os
from Crypto.Util.strxor import strxor
from Crypto.Cipher import AES as aes

In [ ]:
gdown.download_folder('https://drive.google.com/drive/folders/1ziT1Z1O-CGzrj-QSSaLVSHlzL8Bs1730?usp=sharing')

Retrieving folder list


Processing file 1DY-iepTkucfgZCyKUktYmc_yjrBZABQj victoria_1KB.txt
Processing file 1WzXmNu8gUgyTxbwOjqIYSl52Np4S0cE5 victory_5KB.txt
Processing file 1km818Ld_lNtDx-6uoovQL75vj6Zja8Gg victory_10KB.txt
Processing file 1Fc_dv7u6TI8fG5UIboxvd9Afq_5OJx95 victory_100KB.txt
Building directory structure completed


Retrieving folder list completed
Building directory structure
Downloading...
From: https://drive.google.com/uc?id=1DY-iepTkucfgZCyKUktYmc_yjrBZABQj
To: /content/victory/victoria_1KB.txt
100%|██████████| 791/791 [00:00<00:00, 812kB/s]
Downloading...
From: https://drive.google.com/uc?id=1WzXmNu8gUgyTxbwOjqIYSl52Np4S0cE5
To: /content/victory/victory_5KB.txt
100%|██████████| 5.09k/5.09k [00:00<00:00, 9.55MB/s]
Downloading...
From: https://drive.google.com/uc?id=1km818Ld_lNtDx-6uoovQL75vj6Zja8Gg
To: /content/victory/victory_10KB.txt
100%|██████████| 9.61k/9.61k [00:00<00:00, 20.6MB/s]
Downloading...
From: https://drive.google.com/uc?id=1Fc_dv7u6TI8fG5UIboxvd9Afq_5OJx95
To: /content/victory/victory_100KB.txt
100%|██████████| 102k/102k [00:00<00:00, 68.6MB/s]
Download completed


['/content/victory/victoria_1KB.txt',
 '/content/victory/victory_5KB.txt',
 '/content/victory/victory_10KB.txt',
 '/content/victory/victory_100KB.txt']

# Attacking Our OpenPGP Mode



In [ ]:
file_names, file_data = Sandra.load_data('./victory')

# lets use 1k file
plaintext = file_data[-1]

R     = bytes.fromhex('12311498000000000000000000000000')
_key  = bytes.fromhex('10000000111000011000000360000008')
_s    = aes.block_size

enc_dec_pgp = Sandra.AES(_key, Sandra.MODE_OPENPGP, R)

ciphertext = enc_dec_pgp.encrypt(plaintext)

In [ ]:
def oracle(ciphertext, key, b):
  """
    This function acts as our oracle for the attack
    where it does the integrity check and returns its state

    Inputs: ciphertext, key used to encrypt and block size

    outouts: True if ciphertext passes integrity check
             False otherwise
  """
  cipher = aes.new(key, aes.MODE_ECB)

  # decrypt the first block
  C1 = ciphertext[:b]
  R1 = strxor(C1, cipher.encrypt( b'\x00' * b))

  # decrypt the second block(note that its a 2 byte block)
  C2 = ciphertext[b:b+2]
  R2 = strxor(C2, cipher.encrypt(C1)[:2])

  #now, check
  if R2 == R1[-2:]:
    return True
  else:
    return False

In [ ]:
# lets say we know the first two bytes of first block(M1)
M1 = plaintext[:2]

# now, we intercepted some C which is equal to Ek(M) for some unknown K
C = ciphertext

# and we assume a key size of b bytes
b = _s

# since C1, C2 and C3 are part of C
C1 = C[:b]
C2 = C[b : b+2]
C3 = C[b+2 : b+4]

# placeholder for Ek_0_bb
Ek_0_bb = b''

# we can calculate Ek_C1C2_12 by xoring C3 and M1
Ek_C1C2_12 = strxor(C3, M1)

for D in range(2**16):
  # create
  c_dash = C1[2:] + C2 + D.to_bytes(2, 'big') + C[b+2:]
  if oracle(c_dash, _key, _s):
    print('success', D)
    temp = strxor(C2, D.to_bytes(2, 'big'))
    Ek_0_bb = strxor(Ek_C1C2_12, temp)
    print(Ek_0_bb)

success 9455
b'\xab\n'


In [ ]:
# Note that the original paper had a mistake
# where it used Ek(ci) instead of Ek(c(i+2))
# the corrected version of the paper is in the following book page 87
# https://link.springer.com/chapter/10.1007/11693383_4

# lets say we know the first two bytes of msg block(Mi) instead of M1
i = 1
Mi_plus_1 = plaintext[b*i:b*i+2]
C = ciphertext
b = _s

Ci_plus_2 = C[b*(i) + 2  : b*(i+1) + 2]
Ci_plus_3 = C[b*(i+1) + 2  : b*(i+2) + 2]
Ek_0_bb = 0

Ek_Ci_plus_2_12 = strxor(Ci_plus_3[:2], Mi_plus_1)
for D in range(2**16):
  c_dash = Ci_plus_2 + D.to_bytes(2, 'big') + C[b+2:]
  if oracle(c_dash, _key, _s):
    print('success', D)
    temp = strxor(Ek_Ci_plus_2_12, D.to_bytes(2, 'big'))
    Ek_0_bb = strxor(Ci_plus_2[-2:], temp)
    print(Ek_0_bb)
    break

success 21129
b'\xab\n'


In [ ]:
# lets compare with the original Ek(0)_b-1,b
cipher = aes.new(_key, aes.MODE_ECB)
true_Ek_0 = cipher.encrypt(b'\x00' * _s)
print(true_Ek_0[-2], true_Ek_0[-1],  Ek_0_bb[0],  Ek_0_bb[1])

171 10 171 10


In [ ]:
# lets compute the first 2 bytes in all blocks
plaintext_dec = b""

for i in range(len(ciphertext[18:]) // _s):

  Ci_plus_2 = C[b*(i) + 2  : b*(i+1) + 2]
  Ci_plus_3 = C[b*(i+1) + 2: b*(i+2) + 2]

  for D in range(2**16):
    c_dash = Ci_plus_2 + D.to_bytes(2, 'big') + C[b+2:]
    if oracle(c_dash, _key, _s):
      temp = strxor(Ci_plus_2[-2:], D.to_bytes(2, 'big'))
      temp = strxor(Ek_0_bb, temp)
      Mi_plus_1 = strxor(Ci_plus_3[:2], temp)
      plaintext_dec += Mi_plus_1 + b'-' * (b-2)
      break

In [ ]:
print("{:<10} {:<2} {:<10} {:<2} {:<10} {:<2}".format('Block No.', '|', 'Attacked', '|', 'Original', '|'))
print("_"*40)
for i in range(len(ciphertext[18:]) // _s):
  print("{:<10} {:<2} {:<10} {:<2} {:<10} {:<2}".format(i+1, '|', str(plaintext_dec[i*b: i*b + 2]), '|', str(plaintext[i*b: i*b + 2]), '|'))
  print("_"*40)

Block No.  |  Attacked   |  Original   | 
________________________________________
1          |  b'Vi'      |  b'Vi'      | 
________________________________________
2          |  b'nl'      |  b'nl'      | 
________________________________________
3          |  b'ai'      |  b'ai'      | 
________________________________________
4          |  b'e '      |  b'e '      | 
________________________________________
5          |  b'an'      |  b'an'      | 
________________________________________
6          |  b'ce'      |  b'ce'      | 
________________________________________
7          |  b'e '      |  b'e '      | 
________________________________________
8          |  b'th'      |  b'th'      | 
________________________________________
9          |  b'om'      |  b'om'      | 
________________________________________
10         |  b'ts'      |  b'ts'      | 
________________________________________
11         |  b'me'      |  b'me'      | 
________________________________________
12  